# Benchmark of existing approaches for detecting machine-generated text

This notebook enables to evaluate the RoBERTa-based text classifier by Solaiman et al.

## Solaiman: fine-tuned RoBERTa


_Irene Solaiman, Miles Brundage, Jack Clark, Amanda Askell, Ariel Herbert-Voss, Jeff Wu, Alec Radford,
Gretchen Krueger, Jong Wook Kim, Sarah Kreps, Miles McCain, Alex Newhouse, Jason Blazakis, Kris McGuffie, and Jasmine Wang. 2019. Release strategies and the social impacts of language models._

### Install dependencies
To run the code in this notebook, you must install the required dependencies. 

#### Virtual environment 
The suggested approach for installing all required dependencies is by using a corda environment, importing configuration from the `env_transf291.yml` file. 

Create a new environment using conda
``` bash 
conda env create -f env_transf291.yml
```

Activate and install `ipykernel`
```bash
conda activate transf291
conda install ipykernel
```
You should be able to use your conda environent as Jupyter Notebook kernels. 
In case you can't use transf291, you can try by adding it as kernel as follows. 
``` bash 
ipython kernel install --user --name=transf291
```

Start Jupyter Notebook: 
```bash 
jupyter notebook
```

#### Manual installation
Dependencies from: [https://github.com/HendrikStrobelt/detecting-fake-text/blob/master/requirements.txt](detecting-fake-text/requirements.txt).

### Solaiman et al. Code
Implementation of the RoBERTa-based text classifier. 

Logic extracted form `server.py` by Solaiman et al.: [https://github.com/openai/gpt-2-output-dataset/tree/master/detector](https://github.com/openai/gpt-2-output-dataset/tree/master/detector)

In [1]:
import transformers
assert transformers.__version__ == '2.9.1', "Use transformers 2.9.1. It is available in the conda environment transf291"

In [2]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import json
import torch
from urllib.parse import urlparse, unquote
import pandas as pd
import os
import time

Select model to use RoBERTa base or large

In [3]:
model_name = 'roberta-large'

Define model, tokenizer, and basic functions

In [4]:
model = RobertaForSequenceClassification.from_pretrained(model_name)
tokenizer = RobertaTokenizer.from_pretrained(model_name)
device='cuda' if torch.cuda.is_available() else 'cpu'

device = 'cpu'

In [5]:
def evaluate(query):
    tokens = tokenizer.encode(query)
    all_tokens = len(tokens)
    tokens = tokens[:tokenizer.max_len - 2]
    used_tokens = len(tokens)
    tokens = torch.tensor([tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id]).unsqueeze(0)
    mask = torch.ones_like(tokens)

    with torch.no_grad():
        logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
        probs = logits.softmax(dim=-1)

    fake, real = probs.detach().cpu().flatten().numpy().tolist()

    # Original: 
#    return json.dumps(dict(
#         all_tokens=all_tokens,
#         used_tokens=used_tokens,
#         real_probability=real,
#         fake_probability=fake
#     ))

# Changed to return only the binary classification result. 1 if sentence is likely machine-generated.
    return (fake > 0.5, fake, real)

def initialize(checkpoint):
#     if checkpoint.startswith('gs://'):
#         print(f'Downloading {checkpoint}', file=sys.stderr)
#         subprocess.check_output(['gsutil', 'cp', checkpoint, '.'])
#         checkpoint = os.path.basename(checkpoint)
#         assert os.path.isfile(checkpoint)

    print(f'Loading checkpoint from {checkpoint}')
    data = torch.load(checkpoint, map_location='cpu')
    model.load_state_dict(data['model_state_dict'])
    model.eval()

Download finetuned model and load it into RoBERTa

In [6]:
# Make sure that you can run wget on this machine
!wget https://openaipublic.azureedge.net/gpt-2/detector-models/v1/detector-large.pt
initialize('detector-large.pt')

--2023-12-11 14:52:10--  https://openaipublic.azureedge.net/gpt-2/detector-models/v1/detector-large.pt
Risoluzione di openaipublic.azureedge.net (openaipublic.azureedge.net)... 2620:1ec:bdf::43, 2620:1ec:46::43, 13.107.213.43, ...
Connessione a openaipublic.azureedge.net (openaipublic.azureedge.net)|2620:1ec:bdf::43|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 1425743818 (1,3G) [application/octet-stream]
Salvataggio in: «detector-large.pt.3»

detector-large.pt.3 100%[===================>]   1,33G  41,5MB/s    in 31s     

2023-12-11 14:52:42 (43,6 MB/s) - «detector-large.pt.3» salvato [1425743818/1425743818]

Loading checkpoint from detector-large.pt


## Evaluation

This section is devoted to evaluating the Solaiman et al. model under different datasets. 

In [7]:
data_path = "./data/test/"

### Auxiliary functions

In [8]:
import numpy as np
from sklearn.utils import shuffle

def create_dataset(raw_data, is_machine_generated=True):
    X = np.array(raw_data)
    if is_machine_generated:
        y = np.ones_like(X, dtype=int)
    else:
        y = np.zeros_like(X, dtype=int)
    return pd.DataFrame(data={'X':X, 'y':y})

def concat_and_shuffle(datasets):
    return shuffle(pd.concat(datasets)).reset_index()

In [9]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

def safe_macro_f1(y, y_pred):
    """
    Macro-averaged F1, forcing `sklearn` to report as a multiclass
    problem even when there are just two classes. `y` is the list of
    gold labels and `y_pred` is the list of predicted labels.

    """
    return f1_score(y, y_pred, average='macro', pos_label=None)

def safe_accuracy(y, y_pred):
    return accuracy_score(y, y_pred, normalize=True)

def gtc_evaluate(
        dataset,
        model,
        score_func=safe_accuracy):

    # Predictions if we have labels:
    y_ds, preds = model.predict(dataset)
    confusion = confusion_matrix(y_ds, preds)
    score = score_func(y_ds, preds)
        
    # Return the overall scores and other experimental info:
    return {
        'model': model,
#        'labeled_outputs': y_ds, 
#        'predictions': preds, 
        'confusion_matrix': confusion, 
        'score': score }


In [10]:
class Wrapper:
    def __init__(self):
        pass

    def predict(self, dataset, truncateAt=512):
        y_pred = []
        y_dataset = []
        
        totalsize = len(dataset)
        progressupdate = max(min(100, int(totalsize/10)), 1)
        prog = 0
        print(f"progupd: {progressupdate}; m(m(100, {int(totalsize/10)}), 1)")
        for index, row in dataset.iterrows():
            prog+=1
            if not row['X']: 
                print("Skipping sentence because it appears to be empty")
                continue
            try:
                truncated_sentence = row['X'][:truncateAt] if len(row['X']) > truncateAt else row['X']
                (is_generated, fake, real) = evaluate(truncated_sentence)
                y_pred.append(1 if is_generated else 0)     
                y_dataset.append(row['y'])
                
                if (index) % progressupdate == 0:
                    print(f"... progress: {index} ({prog / totalsize})")
            except:
                print(f"Error while processing the query: {row['X']}")
                break
        return (y_dataset, y_pred)

In [11]:
solaiman_detector = Wrapper()

Testing whether the `gtc_evaluate()` works as expected. By default, we use accuracy to measure the score.

In [12]:
def gtc_evaluate_with_checkpoints(dataset, 
        model,
        checkpoint = 10000,
        score_func=safe_accuracy):
    length = len(dataset)
    splits = min(int(length/checkpoint), length) 
    confmatrix_sum = np.zeros((2, 2))
    for i in range(splits + 1):
        dataset_split = dataset[i * checkpoint : (i+1)*checkpoint]
        if dataset_split.empty:
            break
        partial_r = gtc_evaluate(dataset_split, solaiman_detector, score_func=score_func)
        print (f"----- split #{i} (size: {len(dataset_split)}) -----\n {partial_r['confusion_matrix']}\n----- acc: {partial_r['score']} -----")
        confmatrix_sum += partial_r['confusion_matrix']
    print(f"====================================")
    print(f" Final confusion matrix:\n {confmatrix_sum}")
    (tn, fp, fn, tp) = confmatrix_sum.ravel()
    accuracy = (tp +tn) / (tp + tn + fp + fn)
    print(f" Final accuracy: {accuracy}")
    print(f"====================================")

The Solaiman's model warns that sentences should have less than 512 tokens. So, let's try by filtering out senteces with more tokens than that.

### Experiments on WebText + GPT2-xl-1542M data

In [16]:
data_path

'./data/test/'

In [17]:
ds_hwraw_filename = 'webtext.test.human.jsonl'
ds_hw_raw = pd.read_json(os.path.join(data_path, ds_hwraw_filename), lines = True)

In [18]:
ds_mgraw_filename = 'GPT2-xl-1542M.test.machine.jsonl'
ds_mg_raw = pd.read_json(os.path.join(data_path, ds_mgraw_filename), lines = True)

In [19]:
ds_hw_raw

,id,text
0,0,Is this restaurant family-friendly ? Yes No Un...
1,1,Clinton talks about her time of 'reflection' d...
2,2,House Majority Whip Steve Scalise has been dis...
3,3,Insight Course: Lesson 14\n\nControl of the Mi...
4,4,BY JENNIE MCNULTY\n\nLesbian.com\n\nYou know W...
...,...,...
4995,4995,"The week after my 30th birthday, my best frien..."
4996,4996,Downton Abbey doesn't return for its third sea...
4997,4997,Amazon Studios said today that Stephen Root ha...
4998,4998,Everyone likes clean wheels but not everyone h...


In [20]:
ds_mg_raw

,id,text
0,0,What is this?\n\nThis is a neat little plugin ...
1,1,People carry a memorial sign to honor Robert P...
2,2,Google is launching a new scholarship campaign...
3,3,"Authored by: Kristin Scavuzzo, Managing Direct..."
4,4,>at some point the MPP team stop standardizing...
...,...,...
4995,4995,Mac coverage beginning in 1926 at The Michigan...
4996,4996,Martyr Dadabhoyao got a big experience about s...
4997,4997,"""The nation's smugglers have changed their tac..."
4998,4998,The level of British public support for direct...


In [21]:
mg_raw_pd = create_dataset(ds_mg_raw["text"], is_machine_generated=True)
hw_raw_pd = create_dataset(ds_hw_raw["text"], is_machine_generated=False)

In [22]:
mg_raw_pd

,X,y
0,What is this?\n\nThis is a neat little plugin ...,1
1,People carry a memorial sign to honor Robert P...,1
2,Google is launching a new scholarship campaign...,1
3,"Authored by: Kristin Scavuzzo, Managing Direct...",1
4,>at some point the MPP team stop standardizing...,1
...,...,...
4995,Mac coverage beginning in 1926 at The Michigan...,1
4996,Martyr Dadabhoyao got a big experience about s...,1
4997,"""The nation's smugglers have changed their tac...",1
4998,The level of British public support for direct...,1


In [23]:
hw_raw_pd

,X,y
0,Is this restaurant family-friendly ? Yes No Un...,0
1,Clinton talks about her time of 'reflection' d...,0
2,House Majority Whip Steve Scalise has been dis...,0
3,Insight Course: Lesson 14\n\nControl of the Mi...,0
4,BY JENNIE MCNULTY\n\nLesbian.com\n\nYou know W...,0
...,...,...
4995,"The week after my 30th birthday, my best frien...",0
4996,Downton Abbey doesn't return for its third sea...,0
4997,Amazon Studios said today that Stephen Root ha...,0
4998,Everyone likes clean wheels but not everyone h...,0


In [43]:
%%time
gtc_evaluate_with_checkpoints(mg_raw_pd, solaiman_detector, checkpoint=1000, score_func=safe_accuracy)

progupd: 100; m(m(100, 100), 1)
... progress: 0 (0.001)
... progress: 100 (0.101)
... progress: 200 (0.201)
... progress: 300 (0.301)
... progress: 400 (0.401)
... progress: 500 (0.501)
... progress: 600 (0.601)
... progress: 700 (0.701)
... progress: 800 (0.801)
... progress: 900 (0.901)
----- split #0 (size: 1000) -----
 [[  0   0]
 [132 868]]
----- acc: 0.868 -----
progupd: 100; m(m(100, 100), 1)
... progress: 1000 (0.001)
... progress: 1100 (0.101)
... progress: 1200 (0.201)
... progress: 1300 (0.301)
... progress: 1400 (0.401)
... progress: 1500 (0.501)
... progress: 1600 (0.601)
... progress: 1700 (0.701)
... progress: 1800 (0.801)
... progress: 1900 (0.901)
----- split #1 (size: 1000) -----
 [[  0   0]
 [124 876]]
----- acc: 0.876 -----
progupd: 100; m(m(100, 100), 1)
... progress: 2000 (0.001)
... progress: 2100 (0.101)
... progress: 2200 (0.201)
... progress: 2300 (0.301)
... progress: 2400 (0.401)
... progress: 2500 (0.501)
... progress: 2600 (0.601)
... progress: 2700 (0.701

In [44]:
%%time
gtc_evaluate_with_checkpoints(hw_raw_pd, solaiman_detector, checkpoint=1000, score_func=safe_accuracy)

progupd: 100; m(m(100, 100), 1)
... progress: 0 (0.001)
... progress: 100 (0.101)
... progress: 200 (0.201)
... progress: 300 (0.301)
... progress: 400 (0.401)
... progress: 500 (0.501)
... progress: 600 (0.601)
... progress: 700 (0.701)
... progress: 800 (0.801)
... progress: 900 (0.901)
----- split #0 (size: 1000) -----
 [[955  45]
 [  0   0]]
----- acc: 0.955 -----
progupd: 100; m(m(100, 100), 1)
... progress: 1000 (0.001)
... progress: 1100 (0.101)
... progress: 1200 (0.201)
... progress: 1300 (0.301)
... progress: 1400 (0.401)
... progress: 1500 (0.501)
... progress: 1600 (0.601)
... progress: 1700 (0.701)
... progress: 1800 (0.801)
... progress: 1900 (0.901)
----- split #1 (size: 1000) -----
 [[947  53]
 [  0   0]]
----- acc: 0.947 -----
progupd: 100; m(m(100, 100), 1)
... progress: 2000 (0.001)
... progress: 2100 (0.101)
... progress: 2200 (0.201)
... progress: 2300 (0.301)
... progress: 2400 (0.401)
... progress: 2500 (0.501)
... progress: 2600 (0.601)
... progress: 2700 (0.701

### Experiments on GPT3-175b Machine generated text

In [24]:
gpt3_data_path = "./data/test/"
gpt3_ds_mg_filename = 'GPT3-175b.test.machine.jsonl'
with open(os.path.join(gpt3_data_path, gpt3_ds_mg_filename), 'r') as file:
    gpt3_ds_mg = [x for x in file.readlines() if x is not None and x != "\n"]

In [25]:
gpt3_mg_pd = create_dataset(gpt3_ds_mg, is_machine_generated=True)

In [26]:
gpt3_mg_pd

,X,y
0,"{""id"":0,""text"":""Glacier Ridge Christian School...",1
1,"{""id"":1,""text"":""His father was a professor and...",1
2,"{""id"":2,""text"":""Image caption Bailiffs removed...",1
3,"{""id"":3,""text"":""Contents\n\n1. The Meaning of ...",1
4,"{""id"":4,""text"":""Interviews\n\nInterview with A...",1
...,...,...
480,"{""id"":480,""text"":""Get Rangers updates directly...",1
481,"{""id"":481,""text"":""Posted 01 September 2010 - 0...",1
482,"{""id"":482,""text"":""Jeff Chapman\n\nJeffrey Scot...",1
483,"{""id"":483,""text"":""You can look into the Issue ...",1


In [61]:
%%time
gtc_evaluate_with_checkpoints(gpt3_mg_pd, solaiman_detector, checkpoint=1000, score_func=safe_accuracy)

progupd: 4; m(m(100, 4), 1)
... progress: 0 (0.002061855670103093)
... progress: 4 (0.010309278350515464)
... progress: 8 (0.018556701030927835)
... progress: 12 (0.026804123711340205)
... progress: 16 (0.03505154639175258)
... progress: 20 (0.04329896907216495)
... progress: 24 (0.05154639175257732)
... progress: 28 (0.05979381443298969)
... progress: 32 (0.06804123711340206)
... progress: 36 (0.07628865979381444)
... progress: 40 (0.08453608247422681)
... progress: 44 (0.09278350515463918)
... progress: 48 (0.10103092783505155)
... progress: 52 (0.10927835051546392)
... progress: 56 (0.11752577319587629)
... progress: 60 (0.12577319587628866)
... progress: 64 (0.13402061855670103)
... progress: 68 (0.1422680412371134)
... progress: 72 (0.15051546391752577)
... progress: 76 (0.15876288659793814)
... progress: 80 (0.1670103092783505)
... progress: 84 (0.17525773195876287)
... progress: 88 (0.18350515463917524)
... progress: 92 (0.19175257731958764)
... progress: 96 (0.2)
... progress: 

Token indices sequence length is longer than the specified maximum sequence length for this model (1026 > 512). Running this sequence through the model will result in indexing errors


... progress: 188 (0.38969072164948454)
... progress: 192 (0.3979381443298969)
... progress: 196 (0.4061855670103093)
... progress: 200 (0.4144329896907217)
... progress: 204 (0.422680412371134)
... progress: 208 (0.4309278350515464)
... progress: 212 (0.43917525773195876)
... progress: 216 (0.44742268041237115)
... progress: 220 (0.4556701030927835)
... progress: 224 (0.4639175257731959)
... progress: 228 (0.47216494845360824)
... progress: 232 (0.48041237113402063)
... progress: 236 (0.488659793814433)
... progress: 240 (0.49690721649484537)
... progress: 244 (0.5051546391752577)
... progress: 248 (0.51340206185567)
... progress: 252 (0.5216494845360825)
... progress: 256 (0.5298969072164949)
... progress: 260 (0.5381443298969072)
... progress: 264 (0.5463917525773195)
... progress: 268 (0.554639175257732)
... progress: 272 (0.5628865979381443)
... progress: 276 (0.5711340206185567)
... progress: 280 (0.5793814432989691)
... progress: 284 (0.5876288659793815)
... progress: 288 (0.595

### Experiments on GROVER dataset

In [27]:
grover_data_path = "./data/test/"

In [31]:
grover_hw_filename = 'Grover.human.jsonl'
grover_ds_hw = pd.read_json(os.path.join(grover_data_path, grover_ds_filename), lines = True)

#### Mega - p0.94

In [32]:
grover_ds_filename = 'Grover-mega-p0.94.test.machine.jsonl'
grover_ds_mg = pd.read_json(os.path.join(grover_data_path, grover_ds_filename), lines = True)

In [33]:
grover_ds_mg

,id,text
0,1,"When I first started looking at oil assets, I ..."
1,4,The Kingston and St Andrew Municipal Corporati...
2,5,"news, local-news,\nTwo protests against an inv..."
3,7,Drink-driving charges brought against Danny Dr...
4,13,Called by the Mirror after today's strong new ...
...,...,...
9995,24988,"This content was published on April 17, 2019 6..."
9996,24990,"At times like these, it’s hard to know where t..."
9997,24994,The L train is set to shut down for 10 weeks s...
9998,24996,President Donald Trump said on Monday he had o...


In [34]:
grover_ds_hw

,id,text
0,1,"When I first started looking at oil assets, I ..."
1,4,The Kingston and St Andrew Municipal Corporati...
2,5,"news, local-news,\nTwo protests against an inv..."
3,7,Drink-driving charges brought against Danny Dr...
4,13,Called by the Mirror after today's strong new ...
...,...,...
9995,24988,"This content was published on April 17, 2019 6..."
9996,24990,"At times like these, it’s hard to know where t..."
9997,24994,The L train is set to shut down for 10 weeks s...
9998,24996,President Donald Trump said on Monday he had o...


In [35]:
grover_mg_pd = create_dataset(grover_ds_mg["text"], is_machine_generated=True)
grover_hw_pd = create_dataset(grover_ds_hw["text"], is_machine_generated=False)

In [36]:
grover_hw_pd

,X,y
0,"When I first started looking at oil assets, I ...",0
1,The Kingston and St Andrew Municipal Corporati...,0
2,"news, local-news,\nTwo protests against an inv...",0
3,Drink-driving charges brought against Danny Dr...,0
4,Called by the Mirror after today's strong new ...,0
...,...,...
9995,"This content was published on April 17, 2019 6...",0
9996,"At times like these, it’s hard to know where t...",0
9997,The L train is set to shut down for 10 weeks s...,0
9998,President Donald Trump said on Monday he had o...,0


In [89]:
%%time
gtc_evaluate_with_checkpoints(grover_mg_pd, solaiman_detector, checkpoint=2500, score_func=safe_accuracy)

progupd: 25; m(m(100, 25), 1)
... progress: 0 (0.0004)
... progress: 25 (0.0104)
... progress: 50 (0.0204)
... progress: 75 (0.0304)
... progress: 100 (0.0404)
... progress: 125 (0.0504)
... progress: 150 (0.0604)
... progress: 175 (0.0704)
... progress: 200 (0.0804)
... progress: 225 (0.0904)
... progress: 250 (0.1004)
... progress: 275 (0.1104)
... progress: 300 (0.1204)
... progress: 325 (0.1304)
... progress: 350 (0.1404)
... progress: 375 (0.1504)
... progress: 400 (0.1604)
... progress: 425 (0.1704)
... progress: 450 (0.1804)
... progress: 475 (0.1904)
... progress: 500 (0.2004)
... progress: 525 (0.2104)
... progress: 550 (0.2204)
... progress: 575 (0.2304)
... progress: 600 (0.2404)
... progress: 625 (0.2504)
... progress: 650 (0.2604)
... progress: 675 (0.2704)
... progress: 700 (0.2804)
... progress: 725 (0.2904)
... progress: 750 (0.3004)
... progress: 775 (0.3104)
... progress: 800 (0.3204)
... progress: 825 (0.3304)
... progress: 850 (0.3404)
... progress: 875 (0.3504)
...

... progress: 7125 (0.8504)
... progress: 7150 (0.8604)
... progress: 7175 (0.8704)
... progress: 7200 (0.8804)
... progress: 7225 (0.8904)
... progress: 7250 (0.9004)
... progress: 7275 (0.9104)
... progress: 7300 (0.9204)
... progress: 7325 (0.9304)
... progress: 7350 (0.9404)
... progress: 7375 (0.9504)
... progress: 7400 (0.9604)
... progress: 7425 (0.9704)
... progress: 7450 (0.9804)
... progress: 7475 (0.9904)
----- split #2 (size: 2500) -----
 [[   0    0]
 [ 977 1523]]
----- acc: 0.6092 -----
progupd: 25; m(m(100, 25), 1)
... progress: 7500 (0.0004)
... progress: 7525 (0.0104)
... progress: 7550 (0.0204)
... progress: 7575 (0.0304)
... progress: 7600 (0.0404)
... progress: 7625 (0.0504)
... progress: 7650 (0.0604)
... progress: 7675 (0.0704)
... progress: 7700 (0.0804)
... progress: 7725 (0.0904)
... progress: 7750 (0.1004)
... progress: 7775 (0.1104)
... progress: 7800 (0.1204)
... progress: 7825 (0.1304)
... progress: 7850 (0.1404)
... progress: 7875 (0.1504)
... progress: 79

In [90]:
%%time
gtc_evaluate_with_checkpoints(grover_hw_pd, solaiman_detector, checkpoint=2500, score_func=safe_accuracy)

progupd: 25; m(m(100, 25), 1)
... progress: 0 (0.0004)
... progress: 25 (0.0104)
... progress: 50 (0.0204)
... progress: 75 (0.0304)
... progress: 100 (0.0404)
... progress: 125 (0.0504)
... progress: 150 (0.0604)
... progress: 175 (0.0704)
... progress: 200 (0.0804)
... progress: 225 (0.0904)
... progress: 250 (0.1004)
... progress: 275 (0.1104)
... progress: 300 (0.1204)
... progress: 325 (0.1304)
... progress: 350 (0.1404)
... progress: 375 (0.1504)
... progress: 400 (0.1604)
... progress: 425 (0.1704)
... progress: 450 (0.1804)
... progress: 475 (0.1904)
... progress: 500 (0.2004)
... progress: 525 (0.2104)
... progress: 550 (0.2204)
... progress: 575 (0.2304)
... progress: 600 (0.2404)
... progress: 625 (0.2504)
... progress: 650 (0.2604)
... progress: 675 (0.2704)
... progress: 700 (0.2804)
... progress: 725 (0.2904)
... progress: 750 (0.3004)
... progress: 775 (0.3104)
... progress: 800 (0.3204)
... progress: 825 (0.3304)
... progress: 850 (0.3404)
... progress: 875 (0.3504)
...

... progress: 7125 (0.8504)
... progress: 7150 (0.8604)
... progress: 7175 (0.8704)
... progress: 7200 (0.8804)
... progress: 7225 (0.8904)
... progress: 7250 (0.9004)
... progress: 7275 (0.9104)
... progress: 7300 (0.9204)
... progress: 7325 (0.9304)
... progress: 7350 (0.9404)
... progress: 7375 (0.9504)
... progress: 7400 (0.9604)
... progress: 7425 (0.9704)
... progress: 7450 (0.9804)
... progress: 7475 (0.9904)
----- split #2 (size: 2500) -----
 [[2415   85]
 [   0    0]]
----- acc: 0.966 -----
progupd: 25; m(m(100, 25), 1)
... progress: 7500 (0.0004)
... progress: 7525 (0.0104)
... progress: 7550 (0.0204)
... progress: 7575 (0.0304)
... progress: 7600 (0.0404)
... progress: 7625 (0.0504)
... progress: 7650 (0.0604)
... progress: 7675 (0.0704)
... progress: 7700 (0.0804)
... progress: 7725 (0.0904)
... progress: 7750 (0.1004)
... progress: 7775 (0.1104)
... progress: 7800 (0.1204)
... progress: 7825 (0.1304)
... progress: 7850 (0.1404)
... progress: 7875 (0.1504)
... progress: 790

... progress: 14000 (0.6004)
... progress: 14025 (0.6104)
... progress: 14050 (0.6204)
... progress: 14075 (0.6304)
... progress: 14100 (0.6404)
... progress: 14125 (0.6504)
... progress: 14150 (0.6604)
... progress: 14175 (0.6704)
... progress: 14200 (0.6804)
... progress: 14225 (0.6904)
... progress: 14250 (0.7004)
... progress: 14275 (0.7104)
... progress: 14300 (0.7204)
... progress: 14325 (0.7304)
... progress: 14350 (0.7404)
... progress: 14375 (0.7504)
... progress: 14400 (0.7604)
... progress: 14425 (0.7704)
... progress: 14450 (0.7804)
... progress: 14475 (0.7904)
... progress: 14500 (0.8004)
... progress: 14525 (0.8104)
... progress: 14550 (0.8204)
... progress: 14575 (0.8304)
... progress: 14600 (0.8404)
... progress: 14625 (0.8504)
... progress: 14650 (0.8604)
... progress: 14675 (0.8704)
... progress: 14700 (0.8804)
... progress: 14725 (0.8904)
... progress: 14750 (0.9004)
... progress: 14775 (0.9104)
... progress: 14800 (0.9204)
... progress: 14825 (0.9304)
... progress: 